## Import packages

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'  # 安装graphviz的路径，用于模型可视化

In [18]:
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers.core import Dense, Activation, Dropout, Reshape, Flatten, Permute, Lambda
from keras.activations import softmax
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import concatenate, Concatenate, Dot
from keras.layers import GlobalMaxPooling1D, GlobalAveragePooling1D, Input, SpatialDropout1D, Bidirectional, TimeDistributed
from keras.layers.recurrent import LSTM, GRU
from keras import backend as K
from keras.preprocessing import sequence, text
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.utils import plot_model
from keras import optimizers
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

## Hyper parameter setting

In [4]:
token = 'words' # based on words or chars
embed_size = 300 # how big is each word vector
max_features = 20885 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 15 # max number of words in a comment to use
num_rnn_units = 256
num_hidden_units = 200
drop_prob = 0.2
max_norm = 5.0

## File path

In [5]:
TRAIN_PATH = './train.csv'
TEST_PATH = './test.csv'
QUESTION_PATH = './q_no_stopwords.csv'
embed_files = {'words': './word_embed.txt', 'chars': './char_embed.txt'}

## Some helper function

In [6]:
# Get question id from a list. Remove the Q
def get_ids(qids):
    ids = []
    for t_ in qids:
        ids.append(int(t_[1:]))
    return np.asarray(ids)

# Get the text
def get_texts(q_list, question_path=QUESTION_PATH):
    qes = pd.read_csv(question_path)
    ids = get_ids(q_list)
    all_tokens = qes[token]
    texts = [all_tokens[t] for t in ids]
    return texts

## Read the text

#### Train data
split some data for validation

In [7]:
train = pd.read_csv(TRAIN_PATH)
list_train = list(zip(train['q1'], train['q2']))
label_train = train['label']
#print(len(list_train), len(label_train))

X_tra, X_val, y_tra, y_val = train_test_split(list_train, label_train, train_size=0.85, random_state=8, shuffle=True)

# get the text list of question 1 and 2
q1_train = [i[0] for i in X_tra]
text1_train = get_texts(q1_train)
q2_train = [i[1] for i in X_tra]
text2_train = get_texts(q2_train)
q1_val = [i[0] for i in X_val]
text1_val = get_texts(q1_val)
q2_val = [i[1] for i in X_val]
text2_val = get_texts(q2_val)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


#### Test data

In [8]:
test = pd.read_csv(TEST_PATH)
list_test = list(zip(test['q1'], test['q2']))

# get the text list of question 1 and 2
q1_test = [i[0] for i in list_test]
text1_test = get_texts(q1_test)
q2_test = [i[1] for i in list_test]
text2_test = get_texts(q2_test)

## Tokenize

In [9]:
tokenizer = Tokenizer(num_words=max_features, lower=False) # Don't lower the W or L!!!
tokenizer.fit_on_texts(pd.read_csv(QUESTION_PATH)[token])

# train set
tokenized1_train = tokenizer.texts_to_sequences(text1_train)
tokenized2_train = tokenizer.texts_to_sequences(text2_train)
X1_train = pad_sequences(tokenized1_train, maxlen=maxlen)
X2_train = pad_sequences(tokenized2_train, maxlen=maxlen)

# validation set
tokenized1_val = tokenizer.texts_to_sequences(text1_val)
tokenized2_val = tokenizer.texts_to_sequences(text2_val)
X1_val = pad_sequences(tokenized1_val, maxlen=maxlen)
X2_val = pad_sequences(tokenized2_val, maxlen=maxlen)

# test set
tokenized1_test = tokenizer.texts_to_sequences(text1_test)
tokenized2_test = tokenizer.texts_to_sequences(text2_test)
X1_test = pad_sequences(tokenized1_test, maxlen=maxlen)
X2_test = pad_sequences(tokenized2_test, maxlen=maxlen)

## Prepare the pretrained word embedding

In [10]:
def get_coefs(line): return line[0], np.asarray(line[1:], dtype='float32')
embed_file = embed_files[token]
embeddings_index = dict(get_coefs(o.strip().split()) for o in open(embed_file, encoding='utf-8'))
print (len(embeddings_index.items()))
#print (list(embeddings_index.items())[20890])

20891


In [11]:
all_embs = np.hstack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
emb_mean,emb_std

(0.015683081, 1.1956546)

In [12]:
word_index = tokenizer.word_index
embedding_matrix = np.random.normal(emb_mean, emb_std, (max_features+1, embed_size))

for word, i in word_index.items():
    if i > max_features: break
    embedding_vector = embeddings_index.get(word)
    #print (i, word, len(embedding_vector))
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [13]:
embedding_matrix = np.asarray(embedding_matrix, dtype='float32')
print (np.shape(embedding_matrix))

(20886, 300)


## Build the model

In [14]:
from keras import backend as K
K.clear_session()

In [15]:
def time_distributed(input_, layers):
    "Apply a list of layers in TimeDistributed mode"
    out_ = []
    node_ = input_
    for layer_ in layers:
        node_ = TimeDistributed(layer_)(node_)
    out_ = node_
    return out_

In [20]:
i1 = Input(shape=(maxlen,))
i2 = Input(shape=(maxlen,))

# embedding layer
emb = Embedding(max_features+1, embed_size, weights=[embedding_matrix], trainable=False)
inp1 = emb(i1)
inp2 = emb(i2)

# enhance lstm
sd = SpatialDropout1D(drop_prob)
gru_att = Bidirectional(LSTM(num_rnn_units, return_sequences=True, dropout=drop_prob, recurrent_dropout=drop_prob))
inp1 = gru_att(sd(inp1))
inp2 = gru_att(sd(inp2))

# attend
da_layers = [
    Dense(num_hidden_units, activation='relu'),
    Dropout(drop_prob),
    Dense(num_hidden_units, activation='relu'),
    Dropout(drop_prob)]
x1 = time_distributed(inp1, da_layers)
x2 = time_distributed(inp2, da_layers)
attention = Dot(axes=-1)([x1, x2])
w_att_1 = Lambda(lambda x: softmax(x, axis=1))(attention)
w_att_2 = Permute((2,1))(Lambda(lambda x: softmax(x, axis=2))(attention))
x1_aligned = Dot(axes=1)([w_att_1, inp1])
x2_aligned = Dot(axes=1)([w_att_2, inp2])

# compose
x1_combined = Concatenate()([inp1, x2_aligned])
x2_combined = Concatenate()([inp2, x1_aligned])
gru_com = Bidirectional(LSTM(num_rnn_units, return_sequences=True, dropout=drop_prob, recurrent_dropout=drop_prob))
x1_compare = gru_com(x1_combined)
x2_compare = gru_com(x2_combined)

compare_layers = [
    Dense(num_hidden_units, activation='relu'),
    Dropout(drop_prob),
    Dense(num_hidden_units, activation='relu'),
    Dropout(drop_prob)]
x1_compare = time_distributed(x1_compare, compare_layers)
x2_compare = time_distributed(x2_compare, compare_layers)

# aggregate
gmp1 = GlobalMaxPooling1D()(x1_compare)
gap1 = GlobalAveragePooling1D()(x1_compare)
gmp2 = GlobalMaxPooling1D()(x2_compare)
gap2 = GlobalAveragePooling1D()(x2_compare)
    
conc = concatenate([gmp1, gap1, gmp2, gap2])
x = BatchNormalization()(conc)
x = Dense(num_hidden_units, activation='relu')(x)
x = Dropout(drop_prob)(x)
x = BatchNormalization()(x)
x = Dense(num_hidden_units, activation='relu')(x)
x = Dropout(drop_prob)(x)
x = BatchNormalization()(x)
x = Dense(num_hidden_units, activation='relu')(x)
x = Dropout(drop_prob)(x)
x = BatchNormalization()(x)
x = Dense(num_hidden_units, activation='relu')(x)
x = Dropout(drop_prob)(x)
x = BatchNormalization()(x)
x = Dense(1, activation='sigmoid')(x)
model = Model(inputs=[i1, i2], outputs=x)

adam = optimizers.Adam(clipnorm=max_norm)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

model.summary()
plot_model(model, to_file='model16.png', show_shapes=True)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 15)           0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 15)           0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 15, 300)      6265800     input_5[0][0]                    
                                                                 input_6[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_3 (SpatialDro (None, 15, 300)      0           embedding_3[0][0]                
          

## Train the model

In [21]:
cp = ModelCheckpoint(filepath="my_model16.h5", save_best_only=True)
es = EarlyStopping(patience=4)
rp = ReduceLROnPlateau(patience = 0)
hist = model.fit([X1_train, X2_train], y_tra, batch_size = 256, epochs=30, validation_data=([X1_val, X2_val], y_val), callbacks=[cp, es, rp])

Train on 216228 samples, validate on 38158 samples
Epoch 1/30
  8704/216228 [>.............................] - ETA: 19:46 - loss: 0.8273 - acc: 0.5155

KeyboardInterrupt: 

## check the loss curve

In [ ]:
print (hist.history)
%matplotlib inline
plt.figure(1)
plt.plot (hist.history['loss'])
plt.plot (hist.history['val_loss'])

## Load the model

In [ ]:
from keras.models import load_model
model = load_model('my_model16.h5')

## predict the test data

In [ ]:
y_pred = model.predict([X1_test, X2_test], batch_size=1024)

In [ ]:
# To make a submission file 
def make_submission(predict_prob):
    with open('sub16.csv', 'w') as file:
        file.write(str('y_pre') + '\n')
        for line in predict_prob:
            #line = np.clip(line, 0.005, 0.995)
            file.write(str(line[0]) + '\n')
    file.close()
    
make_submission(y_pred)